# MNIST Classifier

<center>
    <img alt="MNIST Handwritten Digits Image" src="https://upload.wikimedia.org/wikipedia/commons/f/f7/MnistExamplesModified.png">
</center>

In this example, we will predict handwritten digits in the [MNIST dataset](https://en.wikipedia.org/wiki/MNIST_database) using a [multi-layer perceptron (MLP)](https://en.wikipedia.org/wiki/Multilayer_perceptron). This is similar to the tutorial notebook, but with an added comparison with a standard model.

Of course, other neural network architectures such as [convolutional neural networks (CNNs)](https://en.wikipedia.org/wiki/Convolutional_neural_network) are better suited for this task, but for this example we will stick with MLPs.

## Setup

First, let's prepare the imports and set the keras backend.

In [1]:
import os
os.environ["KERAS_BACKEND"] = "torch"

In [2]:
import keras
import numpy as np

Define constants relating to the data.

In [3]:
NUM_CLASSES = 10        # 10 distinct classes, 0 to 9
INPUT_SHAPE = (28, 28)  # 28 x 28 greyscale images

Load the data from the `mnist` dataset.

In [4]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

Perform some preprocessing.

In [5]:
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)

## Defining the Model

As mentioned, we will be using a MLP for the model. However, instead of using `keras`'s default `Dense` layer, we will use `keras_mml`'s `DenseMML` layer (which stands for Dense Matrix-Multiplication-less). `DenseMML` is designed to be a direct replacement for `Dense` layers in fully-connected layers, so we don't have to change the architecture of the model much.

In [6]:
import keras_mml

Define the `Sequential` model.

In [7]:
model = keras.Sequential(
    [
        keras.Input(shape=INPUT_SHAPE),
        keras.layers.Flatten(),
        keras_mml.layers.DenseMML(256),
        keras_mml.layers.DenseMML(256),
        keras_mml.layers.DenseMML(256),
        keras.layers.Dense(NUM_CLASSES, activation="softmax"),  # The last layer needs to be `Dense` for the output to work
    ],
    name="Classifier-MML"
)

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [8]:
model.summary()

Model: "Classifier-MML"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ torch_dense_mml (TorchDenseMML) │ (None, 256)            │       200,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ torch_dense_mml_1               │ (None, 256)            │        65,536 │
│ (TorchDenseMML)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ torch_dense_mml_2               │ (None, 256)            │        65,536 │
│ (TorchDenseMML)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 334,346 (1.28 MB)

 Trainable params: 334,346 (1.28 MB)

 Non-trainable params: 0 (0.00 B)

We can now train the model.

In [9]:
model.fit(x_train, y_train, batch_size=128, epochs=20, validation_split=0.1)

Epoch 1/20


422/422 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.2720 - loss: 2.2554 - val_accuracy: 0.2245 - val_loss: 2.1237
Epoch 2/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.2468 - loss: 2.0506 - val_accuracy: 0.3090 - val_loss: 1.9303
Epoch 3/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.3174 - loss: 1.9112 - val_accuracy: 0.3250 - val_loss: 1.8999
Epoch 4/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.3445 - loss: 1.8303 - val_accuracy: 0.3360 - val_loss: 1.8126
Epoch 5/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.3517 - loss: 1.7967 - val_accuracy: 0.2890 - val_loss: 1.8754
Epoch 6/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.3557 - loss: 1.7607 - val_accuracy: 0.4148 - val_loss: 1.6525
Epoch 7/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.3784 - loss: 1.7170 - val_accuracy: 0.4313 - val_loss: 1.6155
Epoch 8/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.4124 - loss: 1.6611 - val_accuracy: 0.403

Once the model is trained, let's evaluate it.

In [10]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 1.0348873138427734
Test accuracy: 0.6672999858856201


Not bad!

## A Comparison - An MLP Using Normal `Dense` Layers

Let's compare our model's performance to a model that uses the regular `Dense` layers.

In [11]:
model = keras.Sequential(
    [
        keras.Input(shape=INPUT_SHAPE),
        keras.layers.Flatten(),
        keras.layers.Dense(256),
        keras.layers.Dense(256),
        keras.layers.Dense(256),
        keras.layers.Dense(NUM_CLASSES, activation="softmax"),
    ],
    name="Classifier-Normal"
)

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [12]:
model.summary()

Model: "Classifier-Normal"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_1 (Flatten)             │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       200,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 335,114 (1.28 MB)

 Trainable params: 335,114 (1.28 MB)

 Non-trainable params: 0 (0.00 B)

We'll train the model using the same `batch_size`, `epochs`, and `validation_split`.

In [13]:
model.fit(x_train, y_train, batch_size=128, epochs=20, validation_split=0.1)

Epoch 1/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8489 - loss: 0.5038 - val_accuracy: 0.9230 - val_loss: 0.2811
Epoch 2/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9026 - loss: 0.3380 - val_accuracy: 0.9282 - val_loss: 0.2650
Epoch 3/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9052 - loss: 0.3277 - val_accuracy: 0.9190 - val_loss: 0.2938
Epoch 4/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9141 - loss: 0.3010 - val_accuracy: 0.9193 - val_loss: 0.2778
Epoch 5/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9133 - loss: 0.3075 - val_accuracy: 0.9265 - val_loss: 0.2584
Epoch 6/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9163 - loss: 0.2962 - val_accuracy: 0.9285 - val_loss: 0.2589
Epoch 7/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9196 - loss: 0.2846 - val_accuracy: 0.9258 - val_loss: 0.2666
Epoch 8/20
422/422 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9171 - loss: 0.2854 - val_accuracy: 0.

Again, we evaluate the model.

In [14]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.29694753885269165
Test accuracy: 0.920199990272522


Notice that the accuracy of the normal model is quite close to the accuracy of the MML model. With a slight decrease in accuracy, the model itself does not use matrix multiplications at all, and also reduces memory usage.